**Sommario**

- [Schema e ciclo di vita del database](#schema-e-ciclo-di-vita-del-database)
  - [Definizione dei modelli](#definizione-dei-modelli)
    - [Constraints (vincoli)](#constraints-vincoli)
    - [Tipi di Dato](#tipi-di-dato)
    - [Nomi consentiti per le tabelle](#nomi-consentiti-per-le-tabelle)
  - [Creazione delle tabelle nel DB con `db.create_all()`](#creazione-delle-tabelle-nel-db-con-dbcreate_all)
    - [Migrazioni](#migrazioni)
- [Operazioni CRUD con Flask-SQLAlchemy](#operazioni-crud-con-flask-sqlalchemy)
  - [`models.py`](#modelspy)
  - [`app.py`](#apppy)
  - [C.R.U.D.](#crud)
    - [CREATE con `db.session.add()`](#create-con-dbsessionadd)
    - [READ con `Model.query`](#read-con-modelquery)
    - [UPDATE con attributi](#update-con-attributi)
    - [DELETE con `db.session.delete()`](#delete-con-dbsessiondelete)
  - [Gestione delle transazioni](#gestione-delle-transazioni)
    - [`db.session.commit()` per salvare definitivamente](#dbsessioncommit-per-salvare-definitivamente)
    - [`db.session.flush()` per salvare ogni tanto durante la sessione](#dbsessionflush-per-salvare-ogni-tanto-durante-la-sessione)
    - [`db.session.rollback()` per annullare manualmente le operazioni](#dbsessionrollback-per-annullare-manualmente-le-operazioni)
    - [Esempio di flush, rollback e commit](#esempio-di-flush-rollback-e-commit)
    - [Uso del contesto con `with`](#uso-del-contesto-con-with)

# Schema e ciclo di vita del database

Prima di iniziare ad eseguire le query sui dati, dobbiamo prima creare il database. Per fare questo dobbiamo innanzitutto definire lo *schema del database*.

Abbiamo già detto che agire sullo schema del database significa creare, modificare o eliminare gli oggetti che compongono il database nel suo complesso. Le prime e più importanti cose che possiamo definire sono le *tabelle* (o *modelli*) e le loro *colonne* (o *campi*).

Queste azioni vanno compiute sia in fase di installazione (inizializzazione) dell'applicazione, sia durante futuri aggiornamenti (migrazioni), che potrebbero richiedere modifiche allo schema del DB, come ad esempio l'aggiunta, la modifica o eliminazione di tabelle e/o colonne.

Come con il *Data Definition Language* (DDL) in SQL abbiamo `CREATE`, `ALTER` e `DROP`, anche l'ORM di SQLAlchemy consente queste operazioni attraverso oggetti e metodi Python.

Inizialmente ci limiteremo a creare le tabelle. Per fare ciò abbiamo `SQLAlchemy.create_all()`.

## Definizione dei modelli

In Flask-SQLAlchemy, i modelli vengono definiti come classi Python che ereditano da `SQLAlchemy.Model`. Ogni classe rappresenta una tabella nel database e ogni attributo della classe rappresenta una colonna nella tabella.

Il termine "modello" deriva dal paradigma [MVC (Model-View-Controller)](https://it.wikipedia.org/wiki/Model-view-controller) nel quale con "modello" si intende una struttura dati in cui vengono immagazzinati i dati e che fornisce delle interfacce ([API](https://it.wikipedia.org/wiki/Application_programming_interface)) per poter interagire con essa.

Esempio di definizione di un modello:

```python
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)
# Se crei app e db in due moduli diversi, puoi fare così:
# db = SQLAlchemy()  # questo in model.py
# db.init_app(app)   # questo in app.py

class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
```


### Constraints (vincoli)

I vincoli sono regole che limitano i dati che possono essere inseriti nelle colonne delle tabelle. I vincoli comuni includono:

- **`primary_key=True`**: Identifica in modo univoco ogni record nella tabella.

- **`unique=True`**: Assicura che tutti i valori in una colonna siano unici.

- **`nullable=False`**: Assicura che una colonna non possa avere valori `Null`.

- **`default='...'`**: Assegna un valore predefinito a una colonna se nessun valore viene specificato. Se omesso, il default è il medesimo del motore usato, SQLite, e quindi `Null`.

- **`CheckConstraint('...', name='...')`**: Assicura che tutti i valori in una colonna soddisfino una condizione specifica.

- **`db.ForeignKey('other_table.id')`**: Un vincolo che crea una relazione tra due tabelle. Il valore deve essere dunque un id dell'altra tabella.

Esempio di utilizzo dei vincoli:

```python
class Corsi(db.Model):
    id = db.Column(
        db.Integer,
        primary_key=True
    )
    nome_corso = db.Column(
        db.String(100),
        nullable=False,
        unique=True)
    descrizione = db.Column(
        db.Text,
        default='N/D'
    )
    durata = db.Column(
        db.Integer,
        CheckConstraint('durata > 0', name='check_durata'),
        nullable=False
    )
```

### Tipi di Dato

Flask-SQLAlchemy supporta diversi tipi di dato, che corrispondono ai tipi di dato del database sottostante, nel nostro caso SQLite. Alcuni tipi di dato comuni includono:

- `db.Integer`: Un numero intero (`INTEGER`).
- `db.Float`: Un numero in virgola mobile (`REAL`).
- `db.String(size)`: Una stringa di lunghezza massima specificata (`VARCHAR` &rarr; `TEXT`).
- `db.Text`: Una stringa di lunghezza variabile (`TEXT`).
- `db.Boolean`: Un valore booleano (True/False) (`BOOLEAN` &rarr; `INTEGER` 0, 1). 
- `db.DateTime`: Una data e ora (`TEXT` timestamp).

Esempio di utilizzo dei tipi di dato:

```python
class Product(db.Model):
    id = db.Column(db.Integer)
    name = db.Column(db.String(100))
    price = db.Column(db.Float)
    in_stock = db.Column(db.Boolean)
```

### Nomi consentiti per le tabelle

I nomi delle tabelle in SQLAlchemy sono derivati automaticamente dai nomi delle classi (es. `Miomodello` diventa `'miomodello'` e `MioModello` diventa `'mio_modello'`), ma possono essere specificati manualmente usando l'attributo `__tablename__`.

I nomi delle tabelle devono seguire le convenzioni del database sottostante, che nel nostro caso è SQLite. In ogni caso, per semplificare, ricordiamo che un nome:

- Deve iniziare con una lettera.
- Può solo contenere lettere, numeri e underscore (_).

Esempio di definizione del nome della tabella:

```python
class Order(db.Model):
    __tablename__ = 'xyz_order'
    ...
```

## Creazione delle tabelle nel DB con `db.create_all()`

Dato che stiamo usando un ORM, non dobbiamo scrivere una query con `CREATE TABLE` come quando abbiamo usato `sqlite3` direttamente.

Dopo aver definito tutti i modelli che rappresentano le tabelle, puoi chiamare `db.create_all()` per creare lo schema delle tabelle nel database. È come eseguire un `CREATE TABLE IF NOT EXISTS` in quanto non aggiorna le tabelle se sono già presenti nel database.

La creazione delle tabelle richiede un contesto dell'applicazione. Poiché solitamente vuoi inizializzare il DB prima di avviare il server, a questo punto non hai ancora un contesto, quindi puoi creane uno manualmente con `with app.app_context():`.

```python
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# 1) COLLEGA SQLALCHEMY A FLASK
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

# 2) DEFINISCI I MODELLI (TABELLE)
class User(db.Model):
    ...
class Film(db.Model):
    ...

with app.app_context():
    # 3) CREA LE TABELLE (se non esistono)
    db.create_all()
```

> **ATTENZIONE**: Se definisci i modelli in altri moduli, devi importarli prima di chiamare `.create_all()`, altrimenti SQLAlchemy non li riconoscerà.

### Migrazioni

 Se modifichi le colonne di un modello, utilizza una libreria di migrazione come [Alembic](https://alembic.sqlalchemy.org/en/latest/) con [Flask-Migrate](https://flask-migrate.readthedocs.io/en/latest/) o [Flask-Alembic](https://flask-alembic.readthedocs.io/en/latest/) per generare degli script che automatizzano le modifiche allo lo schema del database.

# Operazioni CRUD con Flask-SQLAlchemy

Come avete appreso nelle lezioni di SQL, l'acronimo [**CRUD**](https://it.wikipedia.org/wiki/CRUD) sta per CREATE, READ, UPDATE e DELETE, ovvero le quattro operazioni fondamentali che possiamo eseguire sui database.

Vediamo come effettuare queste operazioni con l'ORM di SQLAlchemy.

## `models.py`

Immaginiamo di avere una tabella rappresentata da una classe-modello come questo, nel `models.py`:

```python
# models.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()  # Crea l'istanza di SQLAlchemy

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    age = db.Column(db.Integer)
```

## `app.py`

Per poter svolgere le operazioni CRUD nel file del "controller" `app.py`, dobbiamo prima creare e inizializzare correttamente le istanze di Flask e SQLAlchemy e infine creare le tabelle nel database.

Dopodiché abbiamo due scelte:

- Effettuare le operazioni CRUD prima dell'avvio dell'applicazione Flask. Questo può essere utile per effettuare delle operazioni come l'inizializzazione e il popolamento delle tabelle, azioni di migrazione ecc.

- Effettuare le operazioni CRUD all'interno delle funzioni "endpoint" che gestiscono le route, dunque dopo che l'applicazione e il server Flask sono stati avviati.

```python
# app.py

from flask import Flask
from models import db, User

app = Flask(__name__)  # Crea l'istanza di Flask

app.config.update(
    SQLALCHEMY_DATABASE_URI='sqlite:///db.sqlite3',  # Path al file del DB
)

db.init_app(app)  # Inizializza l'istanza di SQLAlchemy con l'app Flask

...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD
...  # <--- ALL'INTERNO DELLE FUNZIONI DI ENDPOINT
...  # <--- CHE GESTISCONO LE ROUTE.

if __name__ == '__main__':
    with app.app_context():
        db.create_all()  # Crea le tabelle nel database, se necessario

        ...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD PRIMA
        ...  # <--- DELL'AVVIO DELL'APP FLASK. PUO' ESSERE UTILE 
        ...  # <--- PER INIZIALIZZARE E POPOLARE IL DB.

    app.run()
```

## C.R.U.D.

Vediamo come eseguire le quattro classiche operazioni CRUD con Flask-SQLAlchemy.

### CREATE con `db.session.add()`

Creare un nuovo record nel database:

```python
new_user = User(name='Alice', email='alice@flask.sql')
db.session.add(new_user)
db.session.commit()

new_user_id = new_user.id  # es. 10 (ID del record creato)
```

### READ con `Model.query`

Per leggere i record dal database:

```python
the_user = User.query.get(10)
all_users = User.query.all()
some_users = User.query.filter(User.age > 18).order_by(User.name).all()
...
```

> PER APPROFONDIRE: Vedi il notebook [sqlalchemy_READ_legacy.ipynb](sqlalchemy_READ_legacy.ipynb)

### UPDATE con attributi

Per aggiornare un record esistente:

```python
user = User.query.get(10)
user.name = 'Bob'
user.email = 'bob@flask.sql'
db.session.commit()
```

### DELETE con `db.session.delete()`

Per eliminare un record esistente:

```python
user = User.query.get(10)
db.session.delete(user)
db.session.commit()
```

## Gestione delle transazioni

Una transazione è una sequenza di operazioni CRUD che vengono eseguite come un'unità logica di lavoro. Una transazione ha quattro proprietà fondamentali, note come [**ACID**](https://it.wikipedia.org/wiki/ACID):

1. **Atomicità**: Ogni transazione è "tutto o niente". O tutte le operazioni nella transazione vengono completate con successo, oppure nessuna di esse viene applicata.
2. **Consistenza**: Una transazione porta il database da uno stato consistente a un altro stato consistente.
3. **Isolamento**: Le transazioni parallele non interferiscono tra loro.
4. **Durabilità**: Una volta che una transazione è stata committata, i cambiamenti sono permanenti nel database, anche in caso di guasti.

Per approfondire, vedi le sezioni [**Transactions and Connection Management**](https://docs.sqlalchemy.org/en/20/orm/session_transaction.html) e [**Session API**](https://docs.sqlalchemy.org/en/20/orm/session_api.html) nella documentazione ufficiale.


### `db.session.commit()` per salvare definitivamente

Il _**commit**_ dice al database di eseguire e finalizzare tutte le operazioni che sono state accodate nella sessione corrente.

Ecco come funziona nel contesto delle transazioni:

- **Inizio della transazione**: Quando inizi a fare operazioni con `db.session` (ad esempio, `db.session.add()`) per aggiungere un nuovo record, queste operazioni sono accumulate nella sessione ma non vengono ancora applicate al database.

- **Commit della transazione**: Quando chiami `db.session.commit()`, SQLAlchemy invia tutte le operazioni accumulate al database come una singola transazione. Se tutte le operazioni vanno a buon fine, la transazione viene committata e le modifiche diventano permanenti.

- **Rollback in caso di errore durante il commit**: Se durante il commit della transazione si verifica un errore proveniente dal DB, SQLAlchemy esegue automaticamente un rollback, annullando tutte le operazioni fatte nella transazione. Questo mantiene il database in uno stato consistente.

> **ATTENZIONE:** Se si verifica un errore prima del commit, l'utente può decidere se eseguire un rollback manualmente. Solitamente questa è una buona prassi per assicurare un comportamento ACID.

### `db.session.flush()` per salvare ogni tanto durante la sessione

Il _**flush**_ invia tutte le operazioni in sospeso nella sessione al database, ma senza committarle. Serve per sincronizzare la sessione con il database, assicurandosi che le modifiche siano visibili alle successive query nella stessa transazione.


### `db.session.rollback()` per annullare manualmente le operazioni

Il _**rollback**_ annulla tutte le operazioni fatte nella sessione corrente, riportando il database allo stato prima dell'inizio della transazione. Anche le operazioni di flush vengono annullate.

### Esempio di flush, rollback e commit

Ecco un esempio che mostra come utilizzare `flush`, `rollback` e `commit` secondo le buone pratiche:

```python
# Prova:
try:
    new_user = User(name='Bob', email='email@miosito.com')
    db.session.add(new_user)
    db.session.flush()  # Sincronizza la sessione con il database
    # Ora new_user.id è disponibile nell'ORM
    print(f'Nuovo User ID: {new_user.id}')

    # Esegui altre operazioni che potrebbero fallire
    another_user = User.query.get(1)
    another_user.email = 'new_email@example.com'
# Ma in caso di errore:
except Exception as err:
    db.session.rollback()  # Annulla tutte le operazioni, anche quelle flushate
    print(f'Si è verificato un errore: {err}')
# Se non si sono verificati errori:   
else:
    db.session.commit()  # Salva e conferma tutte le operazioni
```

### Uso del contesto con `with`

All'interno di un contesto `with`, se si verifica un errore, viene effettuato il rollback automaticamente e alla fine del blocco `with` viene eseguito il commit.

Per creare un contesto correttamente, dobbiamo usare il metodo `Session.bein()`:

```python
with db.session.begin()
    new_user = User(name='Bob', email='email@miosito.com')
    db.session.add(new_user)
    db.session.flush()  # Sincronizza la sessione con il database
    # Ora new_user.id è disponibile nell'ORM
    print(f'Nuovo User ID: {new_user.id}')

    # Esegui altre operazioni che potrebbero fallire
    another_user = User.query.get(1)
    another_user.email = 'new_email@example.com'
```